In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

In [2]:
df = pd.read_table('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz', nrows=10000, compression='gzip')
df.head()

ParserError: Error tokenizing data. C error: Expected 15 fields in line 5782, saw 22


In [ ]:
df.describe(include='all')

In [ ]:
df['input'] = "TEXT1: " + df.review_headline.fillna('None') + \
    "; TEXT2: " + df.review_body.fillna('None') + \
    "; TEXT3: " + df.product_title.fillna('None') + "; VER: " + df.verified_purchase.fillna('N')
df.input.head()

In [ ]:
!pip install -Uqq datasets

In [ ]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(df[['review_id', 'review_headline', 'review_body', 'product_title', 'star_rating', 'input']])
ds

In [ ]:
model_nm = 'microsoft/deberta-v3-small'
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

In [ ]:
def tok_func(x): return tokz(x['input'])

tok_ds = ds.map(tok_func, batched=True)
tok_ds = tok_ds.rename_columns({'star_rating':'labels'})
row = tok_ds[0]
row['input'], row['input_ids'], row['labels']

In [ ]:
dds = tok_ds.train_test_split(0.30, seed=42)

# Create train, validation and test datasets
train_ds, test_ds = dds["train"], dds["test"]
ds_split_test = test_ds.train_test_split(test_size=0.66)

val_ds, test_ds = ds_split_test["train"], ds_split_test["test"]

(train_ds, val_ds, test_ds)

In [ ]:

def corr(x,y): return np.corrcoef(x,y)[0][1]

def show_corr(df, a, b):
    x,y = df[a],df[b]
    plt.scatter(x,y, alpha=0.5, s=4)
    plt.title(f'{a} vs {b}; r: {corr(x, y):.2f}')
    
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [ ]:
from transformers import TrainingArguments,Trainer

bs = 128
epochs = 4
lr = 8e-5

In [ ]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=False,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=train_ds, eval_dataset=val_ds,
                  tokenizer=tokz, compute_metrics=corr_d)

In [ ]:
train_result = trainer.train()
train_result